In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from matplotlib import font_manager, rc
from sklearn.svm import SVC
from sklearn import svm, metrics

# Load Data

In [2]:
meta = pd.read_csv("C:/Users/ehdgm/Desktop/movie_data.csv", encoding = 'utf-8')
meta.head()

,movieID,movieNm,Opendate,매출액,audiAcc,scrnCnt,showCnt,genre,grade,mainAct,rating,director,result
0,1,명량,2014-07-30,1.360000e+11,17615686,1587,188720,1,8.88,5,3,5,3
1,2,극한직업,2019-01-23,1.400000e+11,16266338,2003,292816,2,9.20,5,3,5,3
2,3,신과함께-죄와 벌,2017-12-20,1.160000e+11,14411782,1912,213320,6,8.73,5,2,5,3
3,4,어벤져스: 엔드게임,2019-04-24,1.220000e+11,13977602,2835,246433,1,9.49,5,2,5,3
4,5,겨울왕국 2,2019-11-21,1.150000e+11,13747792,2648,299334,6,8.95,3,1,5,3


# Data preprocessing

In [3]:
meta=meta[['movieNm', 'scrnCnt', 'showCnt', 'grade', 'mainAct', 'director', 'audiAcc', 'genre', 'rating']]
meta.head()

,movieNm,scrnCnt,showCnt,grade,mainAct,director,audiAcc,genre,rating
0,명량,1587,188720,8.88,5,5,17615686,1,3
1,극한직업,2003,292816,9.20,5,5,16266338,2,3
2,신과함께-죄와 벌,1912,213320,8.73,5,5,14411782,6,2
3,어벤져스: 엔드게임,2835,246433,9.49,5,5,13977602,1,2
4,겨울왕국 2,2648,299334,8.95,3,5,13747792,6,1


In [4]:
meta['addresult']='' #addresult를 추가 (데이터 전처리과정)
meta.describe()

,scrnCnt,showCnt,grade,mainAct,director,audiAcc,genre,rating
count,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,100.000000,100.000000
mean,1252.730000,108847.250000,8.475700,4.080000,3.020000,5.160479e+06,2.800000,2.580000
std,457.739877,60614.421728,0.708064,1.404754,1.786481,3.863848e+06,1.786594,0.793662
min,499.000000,20786.000000,6.440000,1.000000,1.000000,1.857950e+05,1.000000,1.000000
25%,965.500000,58499.000000,8.197500,3.000000,1.000000,2.078964e+06,1.000000,2.000000
50%,1167.000000,97086.500000,8.635000,5.000000,3.000000,4.153674e+06,3.000000,3.000000
75%,1490.250000,149924.250000,8.995000,5.000000,5.000000,7.051065e+06,4.250000,3.000000
max,2835.000000,299334.000000,9.490000,5.000000,5.000000,1.761569e+07,6.000000,4.000000


In [5]:
def movie_result(num):
    if num>=7000000:
        return 3
    elif num>=3000000:
        return 2
    else:
        return 1

In [6]:
def svm_movie_result(num):
    if num==3:
        return 3
    elif num==2:
        return 2
    else:
        return 1

In [7]:
for i in range(len(meta)):
    row=meta.iloc[i, :]
    num=row['audiAcc']
    addresult=movie_result(num)
    row['addresult']=addresult
    meta.iloc[i]=row
    
meta

<ipython-input-7-39f11a924bce>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['addresult']=addresult


,movieNm,scrnCnt,showCnt,grade,mainAct,director,audiAcc,genre,rating,addresult
0,명량,1587,188720,8.88,5,5,17615686,1,3,3
1,극한직업,2003,292816,9.20,5,5,16266338,2,3,3
2,신과함께-죄와 벌,1912,213320,8.73,5,5,14411782,6,2,3
3,어벤져스: 엔드게임,2835,246433,9.49,5,5,13977602,1,2,3
4,겨울왕국 2,2648,299334,8.95,3,5,13747792,6,1,3
...,...,...,...,...,...,...,...,...,...,...
95,연평해전,1013,113767,9.14,1,1,6043784,3,2,2
96,드래곤길들이기2,829,30021,9.26,3,3,3000056,6,1,2
97,군도:민란의 시대,1394,67126,7.34,5,5,4774895,1,3,2
98,슬로우비디오,618,40665,7.77,5,5,1169562,3,2,1


# Statistic Significance

In [8]:
meta["addresult"].value_counts()

1    40
2    34
3    26
Name: addresult, dtype: int64

In [9]:
st=meta.groupby(meta.addresult).mean()
st.columns.name="변수"
st

변수,scrnCnt,showCnt,grade,mainAct,director,audiAcc,genre,rating
addresult,,,,,,,,
1,953.375000,56242.975000,8.069750,3.200000,2.250000,1.828032e+06,3.200000,2.650000
2,1345.970588,117289.470588,8.587647,4.588235,3.117647,4.952425e+06,2.705882,2.617647
3,1591.346154,178737.076923,8.953846,4.769231,4.076923,1.055939e+07,2.307692,2.423077


In [10]:
meta['addresult']=meta['addresult'].astype(str).astype(int)
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieNm    100 non-null    object 
 1   scrnCnt    100 non-null    int64  
 2   showCnt    100 non-null    int64  
 3   grade      100 non-null    float64
 4   mainAct    100 non-null    int64  
 5   director   100 non-null    int64  
 6   audiAcc    100 non-null    int64  
 7   genre      100 non-null    int64  
 8   rating     100 non-null    int64  
 9   addresult  100 non-null    int32  
dtypes: float64(1), int32(1), int64(7), object(1)
memory usage: 7.5+ KB


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets

Y=meta["addresult"] #종속변수
X=meta[["scrnCnt", "showCnt"]] #상영변수의 영향
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

log_clf_1=LogisticRegression() #log_clf_1에 logistic Regression 입력
log_clf_1.fit(X_train, Y_train)
print("예측률: ", log_clf_1.score(X_test, Y_test))

예측률:  0.6333333333333333


In [18]:
Y=meta["addresult"] #종속변수
X=meta[["genre", "rating", "mainAct", "director"]] #제작변수의 영향
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

log_clf_2=LogisticRegression() #log_clf_2에 logistic Regression 입력
log_clf_2.fit(X_train, Y_train) 
print("예측률: ", log_clf_2.score(X_test, Y_test))

예측률:  0.4666666666666667


In [16]:
Y=meta["addresult"] #종속변수
X=meta[["grade"]] #제작변수의 영향
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

log_clf_3=LogisticRegression() #log_clf_3에 logistic Regression 입력
log_clf_3.fit(X_train, Y_train) 
print("예측률: ", log_clf_3.score(X_test, Y_test))

예측률:  0.5


# Data training

In [19]:
x=meta[['scrnCnt','showCnt', 'grade','mainAct', 'director']] #using model_2, linear regression.
y=meta[['audiAcc']]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, test_size=0.3)

mlr=LinearRegression()
mlr.fit(x_train, y_train)

print(x_train.sum()) #학습데이터의 평균을 구하기 위해
print(x_train.mean())
pd.set_option('display.max_rows', 500)
y_predict=mlr.predict(x_test)

print(mlr.score(x_train,y_train))

scrnCnt       85271.00
showCnt     7664232.00
grade           598.33
mainAct         290.00
director        218.00
dtype: float64
scrnCnt       1218.157143
showCnt     109489.028571
grade            8.547571
mainAct          4.142857
director         3.114286
dtype: float64
0.7526667063225416


In [20]:
my_predict=mlr.predict([[1540, 110439, 8.63, 5, 1]])
print("증인의 누적 관객수 예상 : " , my_predict)

증인의 누적 관객수 예상 :  [[4646119.59550918]]


# SVM

In [191]:
svm_data=meta[["grade", "mainAct"]]
svm_result=meta[["addresult"]]
labels=['non', 'fail', 'success', 'big success'] ##1:비흥행, 2 흥행, 3 대흥행

x_train_1, x_test_1, y_train_1, y_test_1=train_test_split(svm_data, svm_result, test_size=0.3, stratify=svm_result)

sc_1=StandardScaler()
sc_1.fit(x_train_1)
x_train_std=sc_1.transform(x_train_1)
x_test_std=sc_1.transform(x_test_1)


model=SVC(kernel='rbf', C=8, gamma=0.1)
model.fit(x_train_std, y_train_1)

print("학습용 데이터셋 정확도: {:.3f}".format(model.score(x_train_std,y_train_1)))
print("검증용 데이터셋 정확도: {:.3f}".format(model.score(x_test_std,y_test_1)))

label_pred=model.predict(x_test_std)

print(model.score(x_test_std, y_test_1))
x_test=[[2648, 299334]]
y_predict=model.predict(x_test)
print(labels[y_predict[0]])

학습용 데이터셋 정확도: 0.586
검증용 데이터셋 정확도: 0.667
0.6666666666666666
fail


C:\Users\ehdgm\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [37]:
#칼럼 자르고 정규화

label=meta["addresult"]
g=meta["scrnCnt"]/2550 #최대 10
m=meta["showCnt"]/300000 #최대5

labels=['non', 'fail', 'success', 'big success'] ##1:비흥행, 2 흥행, 3 대흥행
#정규화 시킨 데이터
gm=pd.concat([g,m], axis=1)

#학습용 데이터 셋과 검증용 데이터셋 분리
data_train, data_test, label_train, label_test=train_test_split(gm, label, test_size=0.3)
#학습
clf=svm.SVC()
clf.fit(data_train, label_train)

#예측
svm_predict=clf.predict(data_test)

print("학습용 데이터셋 정확도: {:.3f}".format(clf.score(data_train, label_train)))
print("검증용 데이터셋 정확도: {:.3f}".format(clf.score(data_test, label_test)))
#print(metrics.classification_report(label_test, svm_predict))

x_test=[[5555, 123123]]
y_predict=clf.predict(x_test)
print(y_predict)


학습용 데이터셋 정확도: 0.829
검증용 데이터셋 정확도: 0.667
[3]


# Final Predict Model

In [42]:
def recommend(scrn, show, grade, actor, director):
    array=['0', 'fail', 'success', 'big success']
    tmp_scrn=scrn/2550
    tmp_show=show/300000
    tmp_gm=[[tmp_scrn,tmp_show]]
    
    tmp_predict_value=clf.predict(tmp_gm)
    
    if tmp_predict_value==3: ##svm 의 예측 정도가 낮으면
        predict_value=mlr.predict([[scrn, show, grade, actor, director]])
        
        predict_value=array[movie_result(predict_value)] #비흥행1, 흥행2, 대흥행3으로 표현 - model 2를 이용한다!
    else:

        predict_value=array[svm_movie_result(tmp_predict_value)]
        
    return predict_value
    
recommend(1540, 110439, 8.63, 5, 1) ##영화 <증인> 관객수 275만명

'success'

In [43]:
recommend(1078, 59626, 7.8, 3, 1) ##테스트데이터, 결과 비흥행과 일치

'fail'